## Автоматизація процесу машинного навчання використовуючи конвеєри (pipelines)

У проекті машинного навчання є стандартні робочі процеси, які можна автоматизувати. В Python scikit-learn, конвеєри (pipelines) допомагають чітко визначити та автоматизувати ці робочі процеси.
* Конвеєри допомагають подолати поширені проблеми, такі як неповнота даних тощо;
* Python scikit-learn надає утиліту Pipeline, яка допомагає автоматизувати робочі процеси машинного навчання;
* Конвеєри працюють, дозволяючи лінійній послідовності перетворень даних зв'язуватись в процес моделювання.

### Підготовка даних та моделювання конвеєру

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Створення pipeline, що стандартизує дані та створює модель
# завантаження бібліотек для обробки даних
import pandas as pd # обробка даних, CSV file I/O (e.g. pd.read_csv)
import numpy as np
from scipy.stats import norm

from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
# візуалізація
import seaborn as sns 
plt.style.use('fivethirtyeight')
sns.set_style("white")

plt.rcParams['figure.figsize'] = (8,4) 
#plt.rcParams['axes.titlesize'] = 'large'

### Оцінка алгоритмів
Потрібно створити деякі моделі даних та оцінити їх точність за невидимими даними. Ось що буде висвітлено на цьому кроці:
1. Відокремлення набору даних перевірки;
2. Налаштування тестового набору для використання 10-кратної крос-валідації;
3. Побудова 5 різних моделей;
4. Вибір найкращої моделі.

### 1.0 Набір даних валідації

In [ ]:
#завантаження даних
data = pd.read_csv('data/clean-data.csv', index_col=False)
data.drop('Unnamed: 0',axis=1, inplace=True)

# розділення набору даних валідації
array = data.values
X = array[:,1:31]
y = array[:,0]

# розділення тестових та навчальних даних
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

# перетворення класових міток з рядкових у цілочисельний тип
le = LabelEncoder()
y = le.fit_transform(y)

### 2.0 Оцінка алгоритмів

In [ ]:
# алгоритми точкової перевірки
models = []
models.append(( 'LR' , LogisticRegression()))
models.append(( 'LDA' , LinearDiscriminantAnalysis()))
models.append(( 'KNN' , KNeighborsClassifier()))
models.append(( 'CART' , DecisionTreeClassifier()))
models.append(( 'NB' , GaussianNB()))
models.append(( 'SVM' , SVC()))

# Варіанти тестування та метрика оцінювання
num_folds = 10
num_instances = len(X_train)
seed = 7 
scoring =  'accuracy'

# Варіанти тестування та метрика оцінювання
num_folds = 10
num_instances = len(X_train)
seed = 7 
scoring =  'accuracy'
results = []
names = []
for name, model in models:
 kfold = KFold(n=num_instances, n_folds=num_folds, random_state=seed)
 cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
 results.append(cv_results)
 names.append(name)
 msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
 print(msg)
print('-> 10-Fold cross-validation accurcay score for the training data for six classifiers')

#### Результати спостережень
> Результати говорять про те, що як логістична регресія, так і LDA може бути вартим подальшого вивчення. Це просто середні значення точності. Завжди розумно дивитися на розподіл значень точності, обчислених у складі перехресної перевірки. Це можна виконати графічно, використовуючи графіки box та whisker.

In [ ]:
# порівняння алгоритмів
fig = plt.figure()
fig.suptitle( 'Algorithm Comparison' )
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

#### Результати спостережень
> Усі класифікатори, окрім SVM, показують аналогічний жорсткий розподіл, що є обнадійливим, бо пропонують низьку дисперсію. Дивують погані результати SVM.
> Можливо, що різноманітний розподіл атрибутів може вплинути на точність таких алгоритмів, як SVM.

### Оцінка алгоритмів: стандартизація даних

In [ ]:
# стандартизація набору даних
pipelines = []
pipelines.append(( 'ScaledLR' , Pipeline([( 'Scaler' , StandardScaler()),( 'LR' ,
    LogisticRegression())])))
pipelines.append(( 'ScaledLDA' , Pipeline([( 'Scaler' , StandardScaler()),( 'LDA' ,
    LinearDiscriminantAnalysis())])))
pipelines.append(( 'ScaledKNN' , Pipeline([( 'Scaler' , StandardScaler()),( 'KNN' ,
    KNeighborsClassifier())])))
pipelines.append(( 'ScaledCART' , Pipeline([( 'Scaler' , StandardScaler()),( 'CART' ,
    DecisionTreeClassifier())])))
pipelines.append(( 'ScaledNB' , Pipeline([( 'Scaler' , StandardScaler()),( 'NB' ,
    GaussianNB())])))
pipelines.append(( 'ScaledSVM' , Pipeline([( 'Scaler' , StandardScaler()),( 'SVM' , SVC())])))

results = []
names = []
for name, model in pipelines:
  kfold = KFold(n=num_instances, n_folds=num_folds, random_state=seed)
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold,
      scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

In [ ]:
# порівняння алгоритмів
fig = plt.figure()
fig.suptitle( 'Scaled Algorithm Comparison' )
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

#### Результати спостережень
> Результати показують, що стандартизація даних підвищила результативність SVM настільки, що він став найточнішим серед усіх алгоритмів.

Отримані результати пропонують розгянути у алгоритми SVM та LDA та LR. Дуже ймовірно, що додаткова їх конфігураці може дати ще більш точні моделі.

### Налаштування алгоритмів

#### Налаштування гіпер-параметрів SVC

In [ ]:
# робимо SVC Pipeline
pipe_svc = Pipeline([('scl', StandardScaler()),
                     ('pca', PCA(n_components=2)),
                     ('clf', SVC(probability=True, verbose=False))])

# підгін параметрів Pipeline до навчальних даних
pipe_svc.fit(X_train, y_train)

#print('--> Fitted Pipeline to training Data')

scores = cross_val_score(estimator=pipe_svc, X=X_train, y=y_train, cv=10, n_jobs=1, verbose=0)
print('--> Model Training Accuracy: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

# налаштування гіпер-параметрів
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [{'clf__C': param_range,'clf__kernel': ['linear']},
              {'clf__C': param_range,'clf__gamma': param_range,
               'clf__kernel': ['rbf']}]
gs_svc = GridSearchCV(estimator=pipe_svc,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=10,
                  n_jobs=1)
gs_svc = gs_svc.fit(X_train, y_train)
print('--> Tuned Parameters Best Score: ',gs.best_score_)
print('--> Best Parameters: \n',gs.best_params_)

### Налаштування гіперпараметрів k-NN
Для стандартної реалізації k-NN, необхідно налаштувати два основні гіпер-параметри:
* кількість сусідів k;
* функція відстаней метрик/подібностей.

Обидва ці значення можуть різко вплинути на точність класифікатора k-NN. Об'єкт Grid може виконати 10-кратну крос-валідацію на моделі KNN, використовуючи класифікаційну точність як метрику оцінювання.
Крім того, за допомогою сітки параметрів можна повторити 10-кратну крос-валідацію 30 разів.
Кожного разу параметру n_neighbors слід надавати інше значення зі списку.
Не можна надати GridSearchCV тільки список.
Необхідно вказати, що n_ouighbors має приймати значення від 1 до 30.
Можна встановити n_jobs = -1 для паралельних обчислень.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN

pipe_knn = Pipeline([('scl', StandardScaler()),
                     ('pca', PCA(n_components=2)),
                     ('clf', KNeighborsClassifier())])
            
# підгін параметрів Pipeline до навчальних даних
pipe_knn.fit(X_train, y_train) 

scores = cross_val_score(estimator=pipe_knn, 
                         X=X_train, 
                         y=y_train, 
                         cv=10,
                         n_jobs=1)
print('--> Model Training Accuracy: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

# налаштування гіперпараметрів
param_range = range(1, 31)
param_grid = [{'clf__n_neighbors': param_range}]
# встановлення сітки
grid = GridSearchCV(estimator=pipe_knn, 
                    param_grid=param_grid, 
                    cv=10, 
                    scoring='accuracy')
gs_knn = grid.fit(X_train, y_train)
print('--> Tuned Parameters Best Score: ',gs.best_score_)
print('--> Best Parameters: \n',gs.best_params_)

#### Допрацювання моделі

In [ ]:
# використання найкращих параметрів
clf_svc = gs.best_estimator_

#Get Final Scores
clf_svc.fit(X_train, y_train)
scores = cross_val_score(estimator=clf_svc,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)
print('--> Final Model Training Accuracy: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

print('--> Final Accuracy on Test set: %.5f' % clf_svc.score(X_test,y_test))

In [ ]:
clf_svc.fit(X_train, y_train)
y_pred = clf_svc.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

### Висновок

Опрацьовано класифікаційне прогностичне моделювання використовуючи машинне навчання за допомогою Python. Зокрема, розглянуті та реалізовані наступні кроки:
1. Визначення проблеми (дані про рак молочної залози);
2. Завантаження вхідного набору даних;
3. Аналіз даних (однаковий масштаб, але різний розподіл даних):
    * оцінка алгоритму (KNN має гарні результати);
    * оцінка алгоритів зі стандартизацією (KNN та SVM мають гарні результати);
4. Налаштування алгоритму (К=19 для KNN мав гарні результати; SVM з RBF ядром та K=100 мав найкращі);
5. Допрацювання моделі (використання усіх навчальних даних та перевірка на тестовому наборі даних).